# Notebook 2: Pre-compute echo-top variable and store back to FDB

In the previous notebooks we demonstrated how users can compute various reprojections and derived data from raw model data. However, this approach may lack efficiency when multiple users or downstream applications need to perform the same Swiss grid reprojection or interpolation to pressure levels on identical data. To optimize the process for widely beneficial and frequently reused data transformations, we aim to pre-compute and **store the results back into FDB** (write access requires user to be in group s83). 

## Configuring Access to FDB

In [ ]:
import os
import logging
import sys
from pathlib import Path

from meteodatalab import mars, mch_model_data


In [ ]:
logging.basicConfig(level=logging.INFO, stream=sys.stdout)
cwd = Path().resolve().parent
view = cwd / "spack-env/.spack-env/view"
assert view.exists()

In [ ]:
os.environ["FDB5_HOME"] = str(view)
os.environ["FDB5_CONFIG"] = """
---
type: remote
engine: remote
host: balfrin-ln003.cscs.ch
port: 30005
store: remote
"""
os.environ["FDB_REMOTE_RETRIEVE_QUEUE_LENGTH"] = "100"


## Querying Data

In [ ]:
date = "20240626"
time = "1200"
request_dbz = mars.Request(
    param="DBZ",
    date=date,
    time=time,
    number=0,
    step=60,
    levtype=mars.LevType.MODEL_LEVEL,
    levelist=tuple(range(1, 81)),
    model=mars.Model.ICON_CH1_EPS,
)
request_hhl = mars.Request(
    param="HHL",
    date=date,
    time=time,
    number=0,
    step=0,
    levtype=mars.LevType.MODEL_LEVEL,
    levelist=tuple(range(1, 82)),
    model=mars.Model.ICON_CH1_EPS,
)


In [ ]:
ds = mch_model_data.get_from_fdb(request_dbz)
ds |= mch_model_data.get_from_fdb(request_hhl)

In [ ]:
hhl = ds["HHL"].squeeze(drop=True)
hhl.sel(z=81).plot()


In [ ]:
dbz = ds["DBZ"]
dbz.sel(z=40).plot()


## Compute "EchoTop" field

The EchoTop is related to the active core of a thundercloud and is defined as the greatest height at which a radar reflectivity of 15 dBZ is measured.

In [ ]:
from meteodatalab.operators.destagger import destagger
from meteodatalab.operators.vertical_interpolation import interpolate_k2any


In [ ]:
hfl = destagger(hhl, "z")
# interpolate height to isosurface of radar reflectivity
echo_top = interpolate_k2any(hfl, "high_fold", dbz, [15.0], hfl)
echo_top.plot()

## Write the field back to FDB

Since `interpolate_k2any` is a generic operator, the metadata has to be updated to maintain consistency with the computed values.

In [ ]:
from meteodatalab import metadata


In [ ]:
# override the GRIB template message metadata
echo_top.attrs |= metadata.override(
    echo_top.message, shortName="ECHOTOPinM", typeOfLevel="echoTopInDBZ"
)


In order to archive back to FDB you need write access to the FDB root directory which is only possible through the designated service account at CSCS.

In [ ]:
mch_model_data.archive_to_fdb(echo_top.rename({"DBZ": "z"}))


## Summary

- compute derived variable from direct model output
- update the metadata
- archive the result back to FDB